In [1]:
import copy
import random
from collections import defaultdict

import numpy as np
import tqdm
from IPython import display
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import jax
import jax.numpy as jnp
import flax.linen as nn
import optax
from utils.splus import splus

In [2]:
class RiverSwim:
    def __init__(self, current='WEAK', seed=1234):
        self.num_states = 6
        self.num_actions = 2  # O <=> LEFT, 1 <=> RIGHT

        # Larger current makes it harder to swim up the river
        self.currents = ['WEAK', 'MEDIUM', 'STRONG']
        assert current in self.currents
        self.current = self.currents.index(current) + 1
        assert self.current in [1, 2, 3]

        # Configure reward function
        R = np.zeros((self.num_states, self.num_actions))
        R[0, 0] = 0.005
        R[5, 1] = 0.6

        # Configure transition function
        T = np.zeros((self.num_states, self.num_actions, self.num_states))

        # Encode initial and rewarding state transitions
        left, right = 0, 1
        T[0, left, 0] = 1.
        T[0, right, 0] = 0.4
        T[0, right, 1] = 0.6

        T[5, left, 4] = 1.
        T[5, right, 5] = 0.6
        T[5, right, 4] = 0.4

        # Encode intermediate state transitions
        for s in range(1, self.num_states - 1):
            # Going left always succeeds
            T[s, left, s - 1] = 1.

            # Going right sometimes succeeds
            T[s, right, s] = 0.6
            T[s, right, s - 1] = 0.05 * self.current
            T[s, right, s + 1] = 0.4 - 0.05 * self.current
            assert np.isclose(np.sum(T[s, right]), 1.)

        self.R = np.array(R)
        self.T = np.array(T)

        # Agent always starts at the opposite end of the river
        self.init_state = 0
        self.curr_state = self.init_state

        self.seed = seed
        random.seed(self.seed)
        np.random.seed(self.seed)

    def get_model(self):
        return copy.deepcopy(self.R), copy.deepcopy(self.T)

    def reset(self):
        return self.init_state

    def step(self, action):
        reward = self.R[self.curr_state, action]
        next_state = np.random.choice(np.arange(self.num_states), p=self.T[self.curr_state, action])
        self.curr_state = next_state
        return reward, next_state


In [3]:
# collect dataset
seed = 10
discount = 0.99
env = RiverSwim(seed=seed)

# uniform behavioral policy
behavioral_policy = np.ones([env.num_states, env.num_actions]) / env.num_actions

dataset = defaultdict(list)

# dataset size = 100K
num_episodes = 1000
max_episode_steps = 101
for _ in tqdm.trange(num_episodes):
    traj_dataset = defaultdict(list)

    obs = env.reset()
    for step in range(max_episode_steps):
        action = np.random.choice(np.arange(env.num_actions), p=behavioral_policy[obs])
        reward, next_obs = env.step(action)
        
        traj_dataset['observations'].append(obs)
        traj_dataset['actions'].append(action)
        traj_dataset['next_observations'].append(next_obs)
        traj_dataset['rewards'].append(reward)
        
        obs = next_obs

    for k, v in traj_dataset.items():
        dataset[k].append(v)

train_dataset = {}
for k, v in dataset.items():
    if 'observations' in k or 'actions' in k:
        dtype = np.int32
    else:
        dtype = np.float32
    arr = np.array(v, dtype=dtype)[:, :-1]
    train_dataset[k] = arr.reshape(-1)
arr = np.array(dataset['actions'], np.int32)[:, 1:]
train_dataset['next_actions'] = arr.reshape(-1)

In [4]:
# value iteration to find the optimal Q
rewards, transition_probs = env.get_model()

opt_q = np.zeros([env.num_states, env.num_actions], dtype=np.float32)
for _ in range(10_000):
  opt_q = rewards + discount * np.einsum('ijk,k->ij', transition_probs, np.max(opt_q, axis=-1))
opt_v = np.max(opt_q, axis=-1)

# deterministic optimal policy
opt_policy = np.zeros([env.num_states, env.num_actions])
opt_policy[np.arange(env.num_states), np.argmax(opt_q, axis=-1)] = 1.0

# value iteration to find the behavioral Q
rewards, transition_probs = env.get_model()

behavioral_q = np.zeros([env.num_states, env.num_actions], dtype=np.float32)
for _ in range(10_000):
  behavioral_q = rewards + discount * np.einsum('ijk,k->ij', transition_probs, np.sum(behavioral_policy * behavioral_q, axis=-1))
behavioral_v = np.sum(behavioral_policy * behavioral_q, axis=-1)

In [5]:
print("optimal q and optimal policy: ")
print(opt_q)
print(opt_policy)

print("behavioral q: ")
print(behavioral_q)

In [14]:
# evaluate the policy
num_episodes = 100
max_episode_steps = 101
returns = []
for _ in tqdm.trange(num_episodes):
    traj_dataset = defaultdict(list)

    obs = env.reset()
    for step in range(max_episode_steps):
        action = np.random.choice(np.arange(env.num_actions), p=opt_policy[obs])
        reward, next_obs = env.step(action)
        
        traj_dataset['observations'].append(obs)
        traj_dataset['actions'].append(action)
        traj_dataset['next_observations'].append(next_obs)
        traj_dataset['rewards'].append(reward)
        
        obs = next_obs

    returns.append(np.sum(traj_dataset['rewards']))

returns = np.mean(returns)
print("returns = {}".format(returns))


#### Utils

In [6]:
def sample_batch(batch_size):
    dataset_size = len(train_dataset['observations'])
    idxs = np.random.randint(dataset_size, size=batch_size)
    batch = jax.tree_util.tree_map(lambda arr: arr[idxs], train_dataset)
    
    return batch

In [7]:
def plot_metrics(metrics, logyscale_stats=[], title=''):
  # learning curves
  nrows = np.ceil(len(metrics) / 4).astype(int)
  ncols = 4
  f, axes = plt.subplots(nrows=nrows, ncols=ncols)
  if nrows == 1:
    axes = np.array([axes])
  f.set_figheight(3 * nrows)
  f.set_figwidth(3 * ncols)

  for idx, (name, val) in enumerate(metrics.items()):
    v = np.array(val)
    if len(v) == 0:
      continue

    x, y = v[:, 0], v[:, 1]
    ax = axes[idx // 4, idx % 4]

    if 'train' in name:
      y = gaussian_filter1d(y, 100)
    ax.plot(x, y)
    if name in logyscale_stats:
      ax.set_yscale('log')
    ax.set_title(name)

    ax.grid()

  f.suptitle(title)

  return f

#### Q-learning

In [15]:
class Critic(nn.Module):
  @nn.compact
  def __call__(self, obs, action):
    obs = jax.nn.one_hot(obs, env.num_states)
    action = jax.nn.one_hot(action, env.num_actions)
    inputs = jnp.concatenate([obs, action], axis=-1)

    qs = nn.Sequential([
      nn.Dense(512),
      nn.gelu,
      # nn.LayerNorm(),
      nn.Dense(512),
      nn.gelu,
      # nn.LayerNorm(),
      nn.Dense(1),
    ])(inputs)
    
    qs = qs.squeeze(-1)

    return qs

In [19]:
batch_size = 1024
tau = 0.005
num_iterations = 50_000
eval_interval = 5_000
log_interval = 5_000

key = jax.random.PRNGKey(np.random.randint(0, 2**32))
key, critic_key = jax.random.split(key)

example_batch = sample_batch(2)
critic = Critic()
critic_params = critic.init(critic_key, example_batch['observations'], example_batch['actions'])
target_critic_params = copy.deepcopy(critic_params)

def loss_fn(params, target_params, batch):
  q = critic.apply(params, batch['observations'], batch['actions'])
  
  next_q1 = critic.apply(target_params, batch['next_observations'], jnp.zeros_like(batch['next_observations']))
  next_q2 = critic.apply(target_params, batch['next_observations'], jnp.ones_like(batch['next_observations']))
  next_q = jnp.maximum(next_q1, next_q2)
  target_q = batch['rewards'] + discount * next_q

  loss = jnp.mean((q - target_q) ** 2)
  
  info = {
    'loss': loss,
    'q': q.mean(),
  }
  
  return loss, info

# optimizer = splus(learning_rate=3e-4 * 512 * 2)
optimizer = optax.adam(learning_rate=5e-4)
opt_state = optimizer.init(critic_params)
grad_fn = jax.value_and_grad(loss_fn, has_aux=True)

@jax.jit
def update_fn(params, target_params, opt_state, batch):
  (loss, info), grads = grad_fn(params, target_params, batch)
  updates, opt_state = optimizer.update(grads, opt_state, params)
  params = optax.apply_updates(params, updates)
  
  target_params = jax.tree_util.tree_map(
    lambda p, tp: p * tau + tp * (1 - tau),
    params, target_params,
  )
  
  return params, target_params, opt_state, loss, info

def evaluate_fn(params):
  obs = jnp.arange(env.num_states)[:, None].repeat(env.num_actions, axis=1).reshape(-1)
  actions = jnp.arange(env.num_actions)[None, :].repeat(env.num_states, axis=0).reshape(-1)
  q = critic.apply(params, obs, actions)
  q = q.reshape([env.num_states, env.num_actions])
  # scaled_q = q * (opt_q / q).mean()
  
  slopes, intercepts = jnp.linalg.lstsq(
    jnp.stack([q.reshape(-1), jnp.ones_like(q.reshape(-1))], axis=1), 
    opt_q.reshape(-1),
  )[0]
  lstsq_q = slopes * q + intercepts
  
  q_err_mean = jnp.mean(np.abs(q - opt_q))
  # scaled_q_err_mean = jnp.mean(np.abs(scaled_q - opt_q))
  lstsq_q_err_mean = jnp.mean(np.abs(lstsq_q - opt_q))
  q_corr_coef = jnp.corrcoef(q.reshape(-1), opt_q.reshape(-1))[0, 1]
  
  # evaluate the policy
  a = jnp.argmax(q, axis=-1)
  policy = jax.nn.one_hot(a, env.num_actions)
  policy = np.asarray(policy)
  
  eval_env = RiverSwim(seed=seed)
  num_episodes = 100
  max_episode_steps = 101
  returns = []
  for _ in tqdm.trange(num_episodes):
      traj_dataset = defaultdict(list)

      obs = eval_env.reset()
      for _ in range(max_episode_steps):
          action = np.random.choice(np.arange(eval_env.num_actions), p=policy[obs])
          reward, next_obs = eval_env.step(action)
          
          traj_dataset['observations'].append(obs)
          traj_dataset['actions'].append(action)
          traj_dataset['next_observations'].append(next_obs)
          traj_dataset['rewards'].append(reward)
          
          obs = next_obs

      returns.append(np.sum(traj_dataset['rewards']))

  returns = np.mean(returns)
  
  info = {
    'q_err_mean': q_err_mean,
    'lstsq_q_err_mean': lstsq_q_err_mean,
    'q_corr_coef': q_corr_coef,
    'returns': returns
  }
  
  return info

metrics = defaultdict(list)
for i in tqdm.trange(1, num_iterations + 1):
  batch = sample_batch(batch_size)
  critic_params, target_critic_params, opt_state, loss, info = update_fn(
    critic_params, target_critic_params, opt_state, batch)

  for k, v in info.items():
    metrics['train/' + k].append(
      np.array([i, v])
    )

  if i == 1 or i % eval_interval == 0:
    eval_info = evaluate_fn(critic_params)
    for k, v in eval_info.items():
      metrics['eval/' + k].append(
        np.array([i, v])
      )

  if i == 1 or i % log_interval == 0:
    plot_metrics(metrics, logyscale_stats=['eval/q_err_mean', 'eval/lstsq_q_err_mean', 'eval/q_corr_coef'])
    display.clear_output(wait=True)
    plt.tight_layout()
    plt.show()


In [20]:
q_learning_metrics = metrics
print(q_learning_metrics['eval/q_err_mean'][-1])
print(q_learning_metrics['eval/lstsq_q_err_mean'][-1])
print(q_learning_metrics['eval/q_corr_coef'][-1])
print(q_learning_metrics['eval/returns'][-1])

#### C51

In [23]:
class CategoricalCritic(nn.Module):
  atoms: jnp.ndarray
  
  @nn.compact
  def __call__(self, obs, action):
    obs = jax.nn.one_hot(obs, env.num_states)
    action = jax.nn.one_hot(action, env.num_actions)
    inputs = jnp.concatenate([obs, action], axis=-1)

    num_atoms = len(self.atoms)

    logits = nn.Sequential([
      nn.Dense(512),
      nn.gelu,
      # nn.LayerNorm(),
      nn.Dense(512),
      nn.gelu,
      # nn.LayerNorm(),
      nn.Dense(num_atoms),
    ])(inputs)
    
    logits = logits.reshape([obs.shape[0], num_atoms])
    probs = jax.nn.softmax(logits, axis=-1)
    qs = jnp.sum(probs * self.atoms[None], axis=-1)

    return qs, probs

In [24]:
batch_size = 1024
tau = 0.005
num_atoms = 51
num_iterations = 50_000
eval_interval = 5_000
log_interval = 5_000

rewards, _ = env.get_model()
v_min = rewards.min() / (1 - discount)
v_max = rewards.max() / (1 - discount)
atoms = jnp.linspace(v_min, v_max, num_atoms)
delta_atom = (v_max - v_min) / (num_atoms - 1)
offset = jnp.arange(batch_size, dtype=jnp.int32) * num_atoms
offset = offset[:, None]
offset = jnp.broadcast_to(offset, (batch_size, num_atoms))

key = jax.random.PRNGKey(np.random.randint(0, 2 ** 32))
key, critic_key = jax.random.split(key)

example_batch = sample_batch(2)
cate_critic = CategoricalCritic(atoms)
cate_critic_params = cate_critic.init(critic_key, example_batch['observations'], example_batch['actions'])
target_cate_critic_params = copy.deepcopy(cate_critic_params)

def loss_fn(params, target_params, batch):
  q, probs = cate_critic.apply(params, batch['observations'], batch['actions'])

  next_q1, next_probs1 = cate_critic.apply(target_params, batch['next_observations'], jnp.zeros_like(batch['next_observations']))
  next_q2, next_probs2 = cate_critic.apply(target_params, batch['next_observations'], jnp.ones_like(batch['next_observations']))
  next_q = jnp.stack([next_q1, next_q2], axis=1)
  next_probs = jnp.stack([next_probs1, next_probs2], axis=1)
  next_actions = jnp.argmax(next_q, axis=1)
  next_probs = jnp.sum(next_probs * jax.nn.one_hot(next_actions, env.num_actions)[..., None], axis=1)

  projected_atoms = jnp.clip(batch['rewards'][:, None] + discount * atoms[None], v_min, v_max)
  projected_bins = (projected_atoms - v_min) / delta_atom
  lower_bins = jnp.floor(projected_bins).astype(jnp.int32)
  upper_bins = jnp.ceil(projected_bins).astype(jnp.int32)

  delta_mass_lower = (upper_bins + (lower_bins == upper_bins).astype(jnp.int32) - projected_bins) * next_probs
  delta_mass_upper = (projected_bins - lower_bins) * next_probs
  
  mass = jnp.zeros(batch_size * num_atoms)
  mass = mass.at[(lower_bins + offset).ravel()].add(delta_mass_lower.ravel())
  mass = mass.at[(upper_bins + offset).ravel()].add(delta_mass_upper.ravel())
  mass = mass.reshape(batch_size, num_atoms)

  loss = jnp.sum(-mass * jnp.log(probs + 1e-8), axis=-1).mean()
  
  # logging
  # q = jnp.sum(qs * jax.nn.one_hot(batch['actions'], env.num_actions), axis=1)
  
  info = {
    'loss': loss,
    'q': q.mean(),
  }
  
  return loss, info

# optimizer = splus(learning_rate=3e-4 * 512 * 2)
optimizer = optax.adam(learning_rate=5e-4)
opt_state = optimizer.init(cate_critic_params)
grad_fn = jax.value_and_grad(loss_fn, has_aux=True)

@jax.jit
def update_fn(params, target_params, opt_state, batch):
  (loss, info), grads = grad_fn(params, target_params, batch)
  updates, opt_state = optimizer.update(grads, opt_state, params)
  params = optax.apply_updates(params, updates)
  
  target_params = jax.tree_util.tree_map(
    lambda p, tp: p * tau + tp * (1 - tau),
    params, target_params,
  )
  
  return params, target_params, opt_state, loss, info

def evaluate_fn(params):
  obs = jnp.arange(env.num_states)[:, None].repeat(env.num_actions, axis=1).reshape(-1)
  actions = jnp.arange(env.num_actions)[None, :].repeat(env.num_states, axis=0).reshape(-1)
  q, _ = cate_critic.apply(params, obs, actions)
  q = q.reshape([env.num_states, env.num_actions])
  # scaled_q = q * (opt_q / q).mean()
  
  slopes, intercepts = jnp.linalg.lstsq(
    jnp.stack([q.reshape(-1), jnp.ones_like(q.reshape(-1))], axis=1), 
    opt_q.reshape(-1),
  )[0]
  lstsq_q = slopes * q + intercepts
  
  q_err_mean = jnp.mean(np.abs(q - opt_q))
  lstsq_q_err_mean = jnp.mean(np.abs(lstsq_q - opt_q))
  q_corr_coef = jnp.corrcoef(q.reshape(-1), opt_q.reshape(-1))[0, 1]
  
  # evaluate the policy
  a = jnp.argmax(q, axis=-1)
  policy = jax.nn.one_hot(a, env.num_actions)
  policy = np.asarray(policy)
  
  eval_env = RiverSwim(seed=seed)
  num_episodes = 100
  max_episode_steps = 101
  returns = []
  for _ in tqdm.trange(num_episodes):
      traj_dataset = defaultdict(list)

      obs = eval_env.reset()
      for _ in range(max_episode_steps):
          action = np.random.choice(np.arange(eval_env.num_actions), p=policy[obs])
          reward, next_obs = eval_env.step(action)
          
          traj_dataset['observations'].append(obs)
          traj_dataset['actions'].append(action)
          traj_dataset['next_observations'].append(next_obs)
          traj_dataset['rewards'].append(reward)
          
          obs = next_obs

      returns.append(np.sum(traj_dataset['rewards']))
  returns = np.mean(returns)
  
  info = {
    'q_err_mean': q_err_mean,
    'lstsq_q_err_mean': lstsq_q_err_mean,
    'q_corr_coef': q_corr_coef,
    'returns': returns,
  }
  
  return info

metrics = defaultdict(list)
for i in tqdm.trange(1, num_iterations + 1):
  batch = sample_batch(batch_size)
  cate_critic_params, target_cate_critic_params, opt_state, loss, info = update_fn(
    cate_critic_params, target_cate_critic_params, opt_state, batch)

  for k, v in info.items():
    metrics['train/' + k].append(
      np.array([i, v])
    )

  if i == 1 or i % eval_interval == 0:
    eval_info = evaluate_fn(cate_critic_params)
    for k, v in eval_info.items():
      metrics['eval/' + k].append(
        np.array([i, v])
      )

  if i == 1 or i % log_interval == 0:
    plot_metrics(metrics, logyscale_stats=['eval/q_err_mean', 'eval/lstsq_q_err_mean', 'eval/q_corr_coef'])
    display.clear_output(wait=True)
    plt.tight_layout()
    plt.show()


In [25]:
c51_metrics = metrics

print(c51_metrics['eval/q_err_mean'][-1])
print(c51_metrics['eval/lstsq_q_err_mean'][-1])
print(c51_metrics['eval/q_corr_coef'][-1])
print(c51_metrics['eval/returns'][-1])

#### Flow distributional Q-learning

In [26]:
class CriticVectorField(nn.Module):
  @nn.compact
  def __call__(self, ret, time, obs, action):
    obs = jax.nn.one_hot(obs, env.num_states)
    action = jax.nn.one_hot(action, env.num_actions)
    inputs = jnp.concatenate([ret, time, obs, action], axis=-1)

    vector_field = nn.Sequential([
      nn.Dense(512),
      nn.gelu,
      # nn.LayerNorm(),
      nn.Dense(512),
      nn.gelu,
      # nn.LayerNorm(),
      nn.Dense(1),
    ])(inputs)
    
    return vector_field

In [ ]:
batch_size = 1024
tau = 0.005
lam = 0.01
num_flow_steps = 10
num_iterations = 50_000
eval_interval = 5_000
log_interval = 5_000

rewards, _ = env.get_model()
min_reward = rewards.min()
max_reward = rewards.max()

key = jax.random.PRNGKey(np.random.randint(0, 2 ** 32))
key, critic_vf_key, target_critic_vf_key = jax.random.split(key, 3)

example_batch = sample_batch(2)
critic_vf = CriticVectorField()

ex_returns = jnp.ones((2, 1), dtype=example_batch['rewards'].dtype)
ex_times = jnp.ones((2, 1), dtype=example_batch['rewards'].dtype)

critic_vf_params = critic_vf.init(critic_vf_key, 
                                  ex_returns, ex_times, 
                                  example_batch['observations'], example_batch['actions'])
target_critic_vf_params = copy.deepcopy(critic_vf_params)

@jax.jit
def compute_flow_returns(
  params,
  noises,
  observations,
  actions,
  init_times=None,
  end_times=None,
):
  """Compute returns from the return flow model using the Euler method."""
  noisy_returns = noises
  if init_times is None:
    init_times = jnp.zeros((*noisy_returns.shape[:-1], 1), dtype=noisy_returns.dtype)
  if end_times is None:
    end_times = jnp.ones((*noisy_returns.shape[:-1], 1), dtype=noisy_returns.dtype)
  step_size = (end_times - init_times) / num_flow_steps

  def func(carry, i):
    """
    carry: (noisy_goals, )
    i: current step index
    """
    (noisy_returns, ) = carry

    times = i * step_size + init_times
    
    # euler method
    vector_field = critic_vf.apply(
      params, noisy_returns, times, observations, actions)
    new_noisy_returns = noisy_returns + step_size * vector_field
    
    # midpoint method
    # mid_vector_field = critic_vf.apply(
    #   params, noisy_returns, times, observations, actions)
    # mid_noisy_returns = noisy_returns + 0.5 * step_size * mid_vector_field
    # mid_times = times + 0.5 * step_size
    # vector_field = critic_vf.apply(
    #   params, mid_noisy_returns, mid_times, observations, actions)
    # new_noisy_returns = noisy_returns + step_size * vector_field
    
    new_noisy_returns = jnp.clip(
      new_noisy_returns,
      min_reward / (1.0 - discount),
      max_reward / (1.0 - discount),
    )

    return (new_noisy_returns, ), None

  # Use lax.scan to do the iteration
  (noisy_returns, ), _ = jax.lax.scan(
      func, (noisy_returns,), jnp.arange(num_flow_steps))
  noisy_returns = jnp.clip(
    noisy_returns,
    min_reward / (1.0 - discount),
    max_reward / (1.0 - discount),
  )

  return noisy_returns

def loss_fn(params, target_params, batch, rng):
  rng, q_noise_rng, noise_rng, time_rng, q_rng = jax.random.split(rng, 5)
  
  q_noises1, q_noises2 = jax.random.uniform(q_noise_rng, (2, batch_size, 1),
                                            minval=min_reward / (1.0 - discount), maxval=max_reward / (1.0 - discount))
  q1 = (q_noises1 + critic_vf.apply(
    target_params, q_noises1, jnp.zeros((batch_size, 1)), 
    batch['observations'], jnp.zeros_like(batch['observations'])))
  q2 = (q_noises2 + critic_vf.apply(
    target_params, q_noises2, jnp.zeros((batch_size, 1)), 
    batch['observations'], jnp.ones_like(batch['observations'])))
  q = jnp.concatenate([q1, q2], axis=1)
  next_actions = jnp.argmax(q, axis=1)
  # next_actions = batch['next_actions']
  
  times = jax.random.uniform(time_rng, (batch_size, 1))
  noises = jax.random.uniform(noise_rng, (batch_size, 1),
                              minval=min_reward / (1.0 - discount), maxval=max_reward / (1.0 - discount))
  next_returns = compute_flow_returns(
    target_params, noises, batch['next_observations'], next_actions)
  returns = jnp.expand_dims(batch['rewards'], axis=-1) + discount * next_returns
  # returns = jnp.clip(
  #   returns,
  #   min_reward / (1.0 - discount),
  #   max_reward / (1.0 - discount),
  # )
  
  rng, noise_rng = jax.random.split(rng)
  noises = jax.random.uniform(noise_rng, (batch_size, 1),
                              minval=min_reward / (1.0 - discount), maxval=max_reward / (1.0 - discount))
  noisy_returns = times * returns + (1.0 - times) * noises
  target_vector_field = returns - noises
  
  vector_field = critic_vf.apply(params, noisy_returns, times, batch['observations'], batch['actions'])
  vector_field_loss = jnp.mean((vector_field - target_vector_field) ** 2)
  
  rng, noise_rng = jax.random.split(rng)
  noises = jax.random.uniform(noise_rng, (batch_size, 1),
                              minval=min_reward / (1.0 - discount), maxval=max_reward / (1.0 - discount))
  noisy_next_returns = compute_flow_returns(
    target_params, noises, batch['next_observations'], next_actions, end_times=times)
  transformed_noisy_returns = jnp.expand_dims(batch['rewards'], axis=-1) + discount * noisy_next_returns
  # transformed_noisy_returns = jnp.clip(
  #   transformed_noisy_returns,
  #   min_reward / (1.0 - discount),
  #   max_reward / (1.0 - discount),
  # )

  vector_field = critic_vf.apply(
    params, transformed_noisy_returns, times, batch['observations'], batch['actions'])
  next_vector_field = critic_vf.apply(
    params, noisy_next_returns, times, batch['next_observations'], next_actions)
  target_vector_field = critic_vf.apply(
    target_params, transformed_noisy_returns, times, batch['observations'], batch['actions'])
  target_next_vector_field = critic_vf.apply(
    target_params, noisy_next_returns, times, batch['next_observations'], next_actions)
  bootstrapped_vector_field_loss = jnp.mean((vector_field - target_next_vector_field) ** 2 +
                                            (next_vector_field - target_vector_field) ** 2)
  
  loss = vector_field_loss + lam * bootstrapped_vector_field_loss
  
  # Additional metrics for logging.
  q_noises = jax.random.uniform(q_rng, (batch_size, 1),
                                minval=min_reward / (1.0 - discount), maxval=max_reward / (1.0 - discount))
  # q = compute_flow_returns(
  #   params, q_noises, batch['observations'], batch['actions'])
  q = (q_noises + critic_vf.apply(
    params, q_noises, jnp.zeros((batch_size, 1)), 
    batch['observations'], batch['actions']))

  info = {
    'loss': loss,
    'vector_field_loss': vector_field_loss,
    'bootstrapped_vector_field_loss': bootstrapped_vector_field_loss,
    'q_mean': q.mean(),
  }
  
  return loss, info

# optimizer = splus(learning_rate=5e-4 * 512 * 2)
optimizer = optax.adam(learning_rate=5e-4)
opt_state = optimizer.init(critic_vf_params)
grad_fn = jax.value_and_grad(loss_fn, has_aux=True)

@jax.jit
def update_fn(params, target_params, opt_state, batch, rng):
  (loss, info), grads = grad_fn(params, target_params, batch, rng)
  updates, opt_state = optimizer.update(grads, opt_state, params)
  params = optax.apply_updates(params, updates)
  
  target_params = jax.tree_util.tree_map(
    lambda p, tp: p * tau + tp * (1 - tau),
    params, target_params,
  )
  
  return params, target_params, opt_state, loss, info

def evaluate_fn(params, rng):
  obs = jnp.arange(env.num_states)[:, None].repeat(env.num_actions, axis=1).reshape(-1)
  actions = jnp.arange(env.num_actions)[None, :].repeat(env.num_states, axis=0).reshape(-1)

  rng, q_rng = jax.random.split(rng)
  q_noises = jax.random.uniform(q_rng, (obs.shape[0], 1),
                                minval=min_reward / (1.0 - discount), maxval=max_reward / (1.0 - discount))
  q = compute_flow_returns(
    params, q_noises, obs, actions)
  # q = (q_noises + critic_vf.apply(
  #   params, q_noises, jnp.zeros((obs.shape[0], 1)), 
  #   obs, actions))
  q = q.reshape(env.num_states, env.num_actions)
  # scaled_q = q * (opt_q / q).mean()
  
  slopes, intercepts = jnp.linalg.lstsq(
    jnp.stack([q.reshape(-1), jnp.ones_like(q.reshape(-1))], axis=1), 
    opt_q.reshape(-1),
  )[0]
  lstsq_q = slopes * q + intercepts
  
  q_err_mean = jnp.mean(np.abs(q - opt_q))
  lstsq_q_err_mean = jnp.mean(np.abs(lstsq_q - opt_q))
  q_corr_coef = jnp.corrcoef(q.reshape(-1), opt_q.reshape(-1))[0, 1]
  
  # evaluate the policy
  a = jnp.argmax(q, axis=-1)
  policy = jax.nn.one_hot(a, env.num_actions)
  policy = np.asarray(policy)
  
  eval_env = RiverSwim(seed=seed)
  num_episodes = 100
  max_episode_steps = 101
  returns = []
  for _ in tqdm.trange(num_episodes):
      traj_dataset = defaultdict(list)

      obs = eval_env.reset()
      for _ in range(max_episode_steps):
          action = np.random.choice(np.arange(eval_env.num_actions), p=policy[obs])
          reward, next_obs = eval_env.step(action)
          
          traj_dataset['observations'].append(obs)
          traj_dataset['actions'].append(action)
          traj_dataset['next_observations'].append(next_obs)
          traj_dataset['rewards'].append(reward)
          
          obs = next_obs

      returns.append(np.sum(traj_dataset['rewards']))
  returns = np.mean(returns)
  
  info = {
    'q_err_mean': q_err_mean,
    'lstsq_q_err_mean': lstsq_q_err_mean,
    'q_corr_coef': q_corr_coef,
    'returns': returns,
  }
  
  return info

metrics = defaultdict(list)
for i in tqdm.trange(1, num_iterations + 1):
  key, train_key = jax.random.split(key)
  batch = sample_batch(batch_size)
  
  critic_vf_params, target_critic_vf_params, opt_state, loss, info = update_fn(
    critic_vf_params, target_critic_vf_params, opt_state, batch, train_key)

  for k, v in info.items():
    metrics['train/' + k].append(
      np.array([i, v])
    )

  if i == 1 or i % eval_interval == 0:
    key, eval_key = jax.random.split(key)
    eval_info = evaluate_fn(critic_vf_params, eval_key)
    for k, v in eval_info.items():
      metrics['eval/' + k].append(
        np.array([i, v])
      )

  if i == 1 or i % log_interval == 0:
    plot_metrics(metrics, logyscale_stats=['eval/q_err_mean', 'eval/lstsq_q_err_mean', 'eval/q_corr_coef'])
    display.clear_output(wait=True)
    plt.tight_layout()
    plt.show()


In [ ]:
# lam = 0.01
fdql_metrics = metrics
print(fdql_metrics['eval/q_err_mean'][-1])
print(fdql_metrics['eval/lstsq_q_err_mean'][-1])
print(fdql_metrics['eval/q_corr_coef'][-1])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 3.2))

metric_name = 'eval/q_err_mean'
ax = axes[0]
for algo, metrics in zip(['Q-learning', 'C51', 'FDQL'], [q_learning_metrics, c51_metrics, fdql_metrics]):
    metrcs_arr = np.asarray(metrics[metric_name])
    l, = ax.plot(metrcs_arr[:, 0], metrcs_arr[:, 1], label=algo, zorder=3)
ax.set_ylabel(r'$| Q - Q^{\star} |$', fontsize=14)
ax.set_xlim([-4e2, 5e4 + 4e2])
ax.set_yscale('log')
ax.legend()
ax.grid(zorder=3)     

metric_name = 'eval/lstsq_q_err_mean'
ax = axes[1]
for algo, metrics in zip(['Q-learning', 'C51', 'FDQL'], [q_learning_metrics, c51_metrics, fdql_metrics]):
    metrcs_arr = np.asarray(metrics[metric_name])
    l, = ax.plot(metrcs_arr[:, 0], metrcs_arr[:, 1], label=algo, zorder=3)
ax.set_ylabel(r'Least Squared $| Q - Q^{\star} |$', fontsize=14)
ax.set_xlim([-4e2, 5e4 + 4e2])
ax.set_yscale('log')
ax.legend()

metric_name = 'eval/returns'
ax = axes[1]
for algo, metrics in zip(['Q-learning', 'C51', 'FDQL'], [q_learning_metrics, c51_metrics, fdql_metrics]):
    metrcs_arr = np.asarray(metrics[metric_name])
    l, = ax.plot(metrcs_arr[:, 0], metrcs_arr[:, 1], label=algo, zorder=3)
ax.set_ylabel(r'returns', fontsize=14)
ax.set_xlim([-4e2, 5e4 + 4e2])
ax.legend()

# ax.set_ylim([-5, 120 + 5])
# ax.set_yticks([0, 120])
# ax.yaxis.set_minor_locator(MultipleLocator(100))
ax.grid(zorder=3)     

fig.tight_layout(rect=(-0.026, -0.06, 1.02, 1.04))  # rect = (left, bottom, right, top), default: (0, 0, 1, 1)
# filepath = "/u/cz8792/research/ogbench/plot_scripts/figures/convergence_speed_ablation_lc.pdf"
# fig.savefig(filepath, dpi=150, bbox_inches="tight")
plt.show()
